In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def get_html(url):
    _html = ""
    resp = requests.get(url)
    if resp.status_code == 200:
        _html = resp.text
    return _html


URL = "http://finance.naver.com/sise/entryJongmok.nhn?&page=1"
html = get_html(URL)
soup = BeautifulSoup(html, 'html.parser')


######## KOSPI 지수 크롤링 ################
target_n=240       ## 크롤링 할 데이터의 갯수

df_index=pd.DataFrame(np.zeros((target_n+6,4)),columns=['Date','Close','Volume','tvalue'])
for i in range(int(target_n/6)+1):
    URL="http://finance.naver.com/sise/sise_index_day.nhn?code=KPI200&page="+str(i+1)
    html=get_html(URL)
    soup=BeautifulSoup(html, 'html.parser')
    meta_date=soup('td',{'class':'date'})
    meta=soup('td',{'class':'number_1'})
    for j in range(len(meta_date)):
        df_index.iloc[i*len(meta_date)+j,0]=str(meta_date[j]).split('"date">')[1].split('</td')[0].replace(',','')
        df_index.iloc[i*len(meta_date)+j,1]=str(meta[4*j]).split('"number_1">')[1].split('</td')[0].replace(',','')
        df_index.iloc[i*len(meta_date)+j,2]=str(meta[4*j+2]).split('40px;">')[1].split('</td')[0].replace(',','')
        df_index.iloc[i*len(meta_date)+j,3]=str(meta[4*j+3]).split('30px;">')[1].split('</td')[0].replace(',','')

df_index.iloc[::-1].set_index('Date').to_csv('KOSPI200_index.csv')


# 1) 종목코드, 종목명 구하기
code_list=[]
name_list=[]
for i in range(1,21):
    URL = "http://finance.naver.com/sise/entryJongmok.nhn?&page="+str(i)
    html = get_html(URL)
    soup = BeautifulSoup(html, 'html.parser')
    
    Code_text = soup('td',{'class':'ctg'},)

    for t in Code_text:
        codeText=str(t)
        code_list.append(codeText.split('code=')[1].split('"')[0])
        name_list.append(codeText.split('parent">')[1].split('</a')[0])

##########2) KOSPI200 종목 크롤링######################

##변수 선언####
target_n=240
adj_code_list=[]
adj_name_list=[]
KOSPI200_close=pd.DataFrame()
##############

##Loop문을 활용한 종목 별 데이터 수집#################
for i in range(len(code_list)):
    data_temp=pd.DataFrame(np.zeros((target_n,2)),columns=['Date','Close'])
    try: #예외 처리 구문
        for j in range(int(target_n/10)):  #한 페이지에 10개씩이므로, 탐색 기간만큼 
            URL="http://finance.naver.com/item/sise_day.nhn?code="+str(code_list[i])+"&page="+str(j+1)
            html=get_html(URL)
            soup=BeautifulSoup(html, 'html.parser')
            meta=soup('tr',{'onmouseover':'mouseOver(this)'})
            for t,tt in enumerate(meta):
                meta_text=str(tt)
                data_temp.iloc[j*10+t,0]=meta_text.split('gray03">')[1].split('</span')[0]
                data_temp.iloc[j*10+t,1]=meta_text.split('p11">')[4].split('</span')[0].replace(',','')
        
        data_temp2=data_temp.iloc[::-1].set_index('Date')
        
        # Close Price만 종목별로 따로 저장
        KOSPI200_close[str(code_list[i])]=data_temp2.iloc[:,0]
        
        #예외 처리에 걸리지 않은 녀석들만 다시 종목코드,종목명 List로 Update
        adj_code_list.append(code_list[i])
        adj_name_list.append(name_list[i])
    except:
        continue

df_code=pd.DataFrame(adj_code_list,columns=['Code'])
df_name=pd.DataFrame(adj_name_list,columns=['Name'])

df_port_info=pd.concat([df_code, df_name], axis=1)
df_port_info.to_csv('codenamelist.csv',encoding='cp949')

KOSPI200_close.to_csv('KOSPI200_close.csv')

In [5]:
# 저장된 데이터 불러오기
pd.read_csv('KOSPI200_close.csv').set_index('Date')

,005930,000660,051910,005380,068270,051900,005490,207940,035420,017670,...,001520,096760,003920,014830,060980,079430,002960,033920,003300,036580
Date,,,,,,,,,,,,,,,,,,,,,
2018.04.04,2346000,80400,376000,156500,294000,1263000,317500,503000,754000,234500,...,1875,8870,686000,49850,49050,25400,340000,17300,106000,12050
2018.04.05,2437000,82900,383500,154000,295000,1282000,318500,502000,779000,233000,...,1860,8830,691000,49600,49250,26650,338000,17400,110000,12200
2018.04.06,2420000,80300,379000,152500,292000,1255000,319000,519000,770000,232500,...,1860,9040,698000,50700,48900,26950,340500,17750,114000,12150
2018.04.09,2460000,80600,376000,151000,306000,1264000,313000,562000,774000,235000,...,1875,9100,698000,50500,48550,26850,339000,17850,114500,12250
2018.04.10,2444000,80400,368000,151000,303000,1305000,324500,583000,775000,232500,...,1870,9160,693000,49800,48500,27200,334500,17650,116000,12050
2018.04.11,2443000,81200,367000,150000,298000,1266000,326500,584000,771000,231000,...,1875,10700,684000,49650,49000,26900,335000,17550,115000,12150
2018.04.12,2450000,84000,378000,152000,298500,1249000,321500,565000,757000,229000,...,1880,9910,680000,49600,48450,26700,337000,17600,119000,12050
2018.04.13,2490000,84200,373000,150500,290000,1276000,327500,570000,764000,224000,...,1875,10000,670000,49100,47550,26500,336000,17250,117000,11800
2018.04.16,2517000,84000,365000,153000,295500,1277000,325000,567000,751000,225500,...,1875,9970,667000,50500,47650,26200,334000,17100,117500,11700
